# 图像爬取

## 1. 导入工具包

In [6]:
# 导入操作系统库以获取平台信息以及对目录操作的权限
import os

# 导入时间库
import time

# 导入网络请求库以支持爬虫操作
import requests

# 导入第三方网络请求库以增强爬虫操作（比标准库内的urllib更强大）
import urllib3
urllib3.disable_warnings() # 禁止urllib3验证https安全证书的警告

# 导入进度条库的模块以令进程可视化（如果仅仅 import tqdm 就需要以 tqdm.tqdm() 的方式来调用）
from tqdm import tqdm

## 2. 设置HTTP请求参数

In [7]:
'''
启动开发人员工具，通过 NetWork — ALL — Name 找到headers（请求头），将里面的内容复制过来(一般反爬有User-Agent就行，不行就把所有的请求头加进去~)
注意：每个都要改写成“key = value”（键值对）形式，可以通过Regex（正则表达式）来修改内容
'''

# 使用字典格式的cookie让request处理，不能放入headers
cookies = {
'BDqhfp': '%E7%8B%97%E7%8B%97%26%26NaN-1undefined%26%2618880%26%2621',
'BIDUPSID': '06338E0BE23C6ADB52165ACEB972355B',
'PSTM': '1646905430',
'BAIDUID': '104BD58A7C408DABABCAC9E0A1B184B4:FG=1',
'BDORZ': 'B490B5EBF6F3CD402E515D22BCDA1598',
'H_PS_PSSID': '35836_35105_31254_36024_36005_34584_36142_36120_36032_35993_35984_35319_26350_35723_22160_36061',
'BDSFRCVID': '8--OJexroG0xMovDbuOS5T78igKKHJQTDYLtOwXPsp3LGJLVgaSTEG0PtjcEHMA-2ZlgogKK02OTH6KF_2uxOjjg8UtVJeC6EG0Ptf8g0M5',
'H_BDCLCKID_SF': 'tJPqoKtbtDI3fP36qR3KhPt8Kpby2D62aKDs2nopBhcqEIL4QTQM5p5yQ2c7LUvtynT2KJnz3Po8MUbSj4QoDjFjXJ7RJRJbK6vwKJ5s5h5nhMJSb67JDMP0-4F8exry523ioIovQpn0MhQ3DRoWXPIqbN7P-p5Z5mAqKl0MLPbtbb0xXj_0D6bBjHujtT_s2TTKLPK8fCnBDP59MDTjhPrMypomWMT-0bFH_-5L-l5js56SbU5hW5LSQxQ3QhLDQNn7_JjOX-0bVIj6Wl_-etP3yarQhxQxtNRdXInjtpvhHR38MpbobUPUDa59LUvEJgcdot5yBbc8eIna5hjkbfJBQttjQn3hfIkj0DKLtD8bMC-RDjt35n-Wqxobbtof-KOhLTrJaDkWsx7Oy4oTj6DD5lrG0P6RHmb8ht59JROPSU7mhqb_3MvB-fnEbf7r-2TP_R6GBPQtqMbIQft20-DIeMtjBMJaJRCqWR7jWhk2hl72ybCMQlRX5q79atTMfNTJ-qcH0KQpsIJM5-DWbT8EjHCet5DJJn4j_Dv5b-0aKRcY-tT5M-Lf5eT22-usy6Qd2hcH0KLKDh6gb4PhQKuZ5qutLTb4QTbqWKJcKfb1MRjvMPnF-tKZDb-JXtr92nuDal5TtUthSDnTDMRhXfIL04nyKMnitnr9-pnLJpQrh459XP68bTkA5bjZKxtq3mkjbPbDfn02eCKuj6tWj6j0DNRabK6aKC5bL6rJabC3b5CzXU6q2bDeQN3OW4Rq3Irt2M8aQI0WjJ3oyU7k0q0vWtvJWbbvLT7johRTWqR4enjb3MonDh83Mxb4BUrCHRrzWn3O5hvvhKoO3MA-yUKmDloOW-TB5bbPLUQF5l8-sq0x0bOte-bQXH_E5bj2qRCqVIKa3f',
'BDSFRCVID_BFESS': '8--OJexroG0xMovDbuOS5T78igKKHJQTDYLtOwXPsp3LGJLVgaSTEG0PtjcEHMA-2ZlgogKK02OTH6KF_2uxOjjg8UtVJeC6EG0Ptf8g0M5',
'H_BDCLCKID_SF_BFESS': 'tJPqoKtbtDI3fP36qR3KhPt8Kpby2D62aKDs2nopBhcqEIL4QTQM5p5yQ2c7LUvtynT2KJnz3Po8MUbSj4QoDjFjXJ7RJRJbK6vwKJ5s5h5nhMJSb67JDMP0-4F8exry523ioIovQpn0MhQ3DRoWXPIqbN7P-p5Z5mAqKl0MLPbtbb0xXj_0D6bBjHujtT_s2TTKLPK8fCnBDP59MDTjhPrMypomWMT-0bFH_-5L-l5js56SbU5hW5LSQxQ3QhLDQNn7_JjOX-0bVIj6Wl_-etP3yarQhxQxtNRdXInjtpvhHR38MpbobUPUDa59LUvEJgcdot5yBbc8eIna5hjkbfJBQttjQn3hfIkj0DKLtD8bMC-RDjt35n-Wqxobbtof-KOhLTrJaDkWsx7Oy4oTj6DD5lrG0P6RHmb8ht59JROPSU7mhqb_3MvB-fnEbf7r-2TP_R6GBPQtqMbIQft20-DIeMtjBMJaJRCqWR7jWhk2hl72ybCMQlRX5q79atTMfNTJ-qcH0KQpsIJM5-DWbT8EjHCet5DJJn4j_Dv5b-0aKRcY-tT5M-Lf5eT22-usy6Qd2hcH0KLKDh6gb4PhQKuZ5qutLTb4QTbqWKJcKfb1MRjvMPnF-tKZDb-JXtr92nuDal5TtUthSDnTDMRhXfIL04nyKMnitnr9-pnLJpQrh459XP68bTkA5bjZKxtq3mkjbPbDfn02eCKuj6tWj6j0DNRabK6aKC5bL6rJabC3b5CzXU6q2bDeQN3OW4Rq3Irt2M8aQI0WjJ3oyU7k0q0vWtvJWbbvLT7johRTWqR4enjb3MonDh83Mxb4BUrCHRrzWn3O5hvvhKoO3MA-yUKmDloOW-TB5bbPLUQF5l8-sq0x0bOte-bQXH_E5bj2qRCqVIKa3f',
'indexPageSugList': '%5B%22%E7%8B%97%E7%8B%97%22%5D',
'cleanHistoryStatus': '0',
'BAIDUID_BFESS': '104BD58A7C408DABABCAC9E0A1B184B4:FG=1',
'BDRCVFR[dG2JNJb_ajR]': 'mk3SLVN4HKm',
'BDRCVFR[-pGxjrCMryR]': 'mk3SLVN4HKm',
'ab_sr': '1.0.1_Y2YxZDkwMWZkMmY2MzA4MGU0OTNhMzVlNTcwMmM2MWE4YWU4OTc1ZjZmZDM2N2RjYmVkMzFiY2NjNWM4Nzk4NzBlZTliYWU0ZTAyODkzNDA3YzNiMTVjMTllMzQ0MGJlZjAwYzk5MDdjNWM0MzJmMDdhOWNhYTZhMjIwODc5MDMxN2QyMmE1YTFmN2QyY2M1M2VmZDkzMjMyOThiYmNhZA==',
'delPer': '0',
'PSINO': '2',
'BA_HECTOR': '8h24a024042g05alup1h3g0aq0q',
}

# 设定在请求过程中传入的头部信息以使浏览器分辨身份
headers = {
'Connection': 'keep-alive',
'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="97", "Chromium";v="97"',
'Accept': 'text/plain, */*; q=0.01',
'X-Requested-With': 'XMLHttpRequest',
'sec-ch-ua-mobile': '?0',
'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36',
'sec-ch-ua-platform': '"macOS"',
'Sec-Fetch-Site': 'same-origin',
'Sec-Fetch-Mode': 'cors',
'Sec-Fetch-Dest': 'empty',
'Referer': 'https://image.baidu.com/search/index?tn=baiduimage&ipn=r&ct=201326592&cl=2&lm=-1&st=-1&fm=result&fr=&sf=1&fmq=1647837998851_R&pv=&ic=&nc=1&z=&hd=&latest=&copyright=&se=1&showtab=0&fb=0&width=&height=&face=0&istype=2&dyTabStr=MCwzLDIsNiwxLDUsNCw4LDcsOQ%3D%3D&ie=utf-8&sid=&word=%E7%8B%97%E7%8B%97',
'Accept-Language': 'zh-CN,zh;q=0.9',
}

## 3. 百度爬图函数(已封装)

In [8]:
'''
Ajax与JSON解析:
当我们以瀑布流的布局查看某个搜图引擎的结果页面时，通常无法一次性看到所有图片(视网站而定)。
接着，随着用户的下拉刷新或向下滚动，浏览器会监听到这一动作，并在滚动条快要达到底部时向服务器请求一个JSON数据，之后将返回的JSON数据再通过浏览器解析出来。
待解析出数据后，浏览器开始动态加载局部这个页面。注意之前加载出来的图片不会消失，而新的图片会添加到网页的尾部。
启动开发人员工具，通过 NetWork — ALL — Name 找到Fetch/XHR，刚打开时是没有显示请求JSON数据的Request URL的，但若接着向下刷新页面，就会显示刷新时请求的Request URL了。
经多次对比发现，每次刷新Request URl只有这几个参数不同:
pn : 起始加载位置数目
rn : 每次加载总数
gsm : 十六进制形式的当前加载起始位置
word : UTF-8编码的关键词参数

百度网页采用的就是瀑布流布局加载——所有图片都是JavaScript解析出来的，也就是说根本没有把图片放在一个个HTML Tag中，因此通过XML节点解析的方法无法获取到图片的URL。
JavaScript中的真实URL一共有三个：thumbURL、middleURL、objURL：
图片质量最佳的是objURL，但采用它作为目标解析时很快会被反爬虫。
而使用thumbURL、middleURL能进行大规模的爬取，但很多图片下载下来无法打开，因此在下载时要跳过这些不正常的URL。
'''

# 定义爬取单个关键词类的函数
def Craw_SingleClass(KEYWORD, DOWNLOAD_NUM = 200): # KEYWORD 检索用的关键词; DOWNLOAD_NUM 拟爬取图像个数
    
    # 若系统路径不存在'DataSet'则新建一个'DataSet'文件夹
    if not os.path.exists('DataSet'):
        os.makedirs('DataSet')
        print('新建DataSet文件夹')

    # 若系统路径已存在'DataSet/%关键词文件夹%'则图片直接保存至该文件夹，否则在'DataSet'下新建一个'%关键词文件夹%'
    if os.path.exists('DataSet/'+KEYWORD):
        print('文件夹 DataSet/{}已存在，之后直接将爬取到的图片保存至该文件夹中'.format(KEYWORD))
    else:
        os.makedirs('DataSet/{}'.format(KEYWORD))
        print('新建文件夹：DataSet/{}'.format(KEYWORD))

    # 设定计数器初始值（1）
    COUNT = 1

    # 当确保进度条库在使用结束时得到了释放
    with tqdm(total = DOWNLOAD_NUM, position = 0, leave = True) as pbar:
        
        # 设定爬取第几张（1）
        NUM = 1
        
        # 设定是否停止爬取（否）
        Break_Flag = False
        
        # 开始爬取
        while not Break_Flag:

            # 设定页数（由于百度图片每次加载30张，故不同的acjson的Request URL对应的pn值以30的量递增）
            PAGE = 30 * COUNT

            # 设定要发送的查询字符串参数（将参数放在url中传递）
            params = (
                ('tn', 'resultjson_com'),
                ('logid', '12508239107856075440'), # ('logid', '12037596196292044001'),
                ('ipn', 'rj'),
                ('ct', '201326592'),
                ('is', ''),
                ('fp', 'result'),
                ('fr', ''),
                ('word', f'{KEYWORD}'),
                ('queryWord', f'{KEYWORD}'),
                ('cl', '2'),
                ('lm', '-1'),
                ('ie', 'utf-8'),
                ('oe', 'utf-8'),
                ('adpicid', ''),
                ('st', '-1'),
                ('z', ''),
                ('ic', ''), # ('ic', '0'),
                ('hd', ''),
                ('latest', ''),
                ('copyright', ''),
                ('s', ''),
                ('se', ''),
                ('tab', ''),
                ('width', ''),
                ('height', ''),
                ('face', '0'),
                ('istype', '2'),
                ('qc', ''),
                ('nc', '1'),
                ('expermode', ''),
                ('nojc', ''),
                ('isAsync', ''),
                ('pn', f'{PAGE}'),
                ('rn', '30'),
                ('gsm', '1e'), # ('gsm', '3c0000000000003c'),
                ('1647838001666', ''), # ('1666085342167', ''),
            )
            
            # 通过requests包向百度图片搜索引擎发送查询字符串（其中acjson为破解百度防爬虫机制的方法）
            response = requests.get('https://image.baidu.com/search/acjson', headers = headers, params = params, cookies = cookies)
            
            # 若向指定URI发出请求时返回响应对象的指示状态为正常(200为正常，404为未找到)
            if response.status_code == 200:

                # 尝试获取返回结果为json格式的链接数据（调用非json格式的结果会返回错误）
                try:
                    json_data = response.json().get("data")

                    # 若获取成功
                    if json_data:

                        # 对JSON数据文件进行遍历
                        for x in json_data:

                            # 解析
                            type = x.get("type") # 获取图像类型
                            if type not in ["gif"]: # 筛选出非gif文件
                                img = x.get("thumbURL") # 如果改为"objURL"（高清原图）很快会被反爬，慎用！
                                fromPageTitleEnc = x.get("fromPageTitleEnc") # 图片搜索结果对应的title  

                                # 尝试移除SSL认证
                                try:
                                    resp = requests.get(url = img, verify = False)

                                    # 睡眠以控制爬虫节奏（防止IP被ban）
                                    time.sleep(1)

                                    # 保存文件
                                    file_save_path = f'DataSet/{KEYWORD}/{NUM}.{type}' # 若想保留图片原标题，在.{type}前加上{fromPageTitleEnc}即可
                                    with open(file_save_path, 'wb') as file: # 以“二进制 写”模式打开文件
                                        file.write(resp.content) # 将HTTP响应内容以二进制(bytes)形式写入
                                        file.flush() # 在关闭文件前刷新缓冲区
                                        NUM += 1
                                        pbar.update(1) # 更新进度条

                                    # 停止爬取
                                    if NUM > DOWNLOAD_NUM: # 若爬取数量已达到要求
                                        if not Break_Flag:
                                            Break_Flag = True
                                        print(f'{NUM - 1}张图像爬取完毕')
                                        break
                                    
                                # 其它异常直接跳过
                                except Exception:
                                    pass
                                
                # 若返回错误直接跳过链接
                except:
                    pass
            
            # 若请求失败终止爬取
            else:
                break

            # 计数器自增（1）
            COUNT += 1

## 4. 关键词输入

- ### 爬取单类

In [ ]:
# 爬取单个关键词类的检索结果，并设定拟爬取图像个数
Craw_SingleClass('%关键词%', DOWNLOAD_NUM = 200)

- ### 爬取多类

In [9]:
# 设定拟爬取的多个关键词类
Class_List = ['%关键词1%','%关键词2%','%关键词3%','%关键词4%','%关键词5%','%关键词6%','%关键词7%','%关键词8%','%关键词9%','%关键词10%','%关键词11%','%关键词12%']
# 对每个关键词类进行爬取，并设定拟爬取图像个数
for Each in Class_List:
    Craw_SingleClass(Each, DOWNLOAD_NUM = 200)